<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [4]:
dataset_path="/content/drive/MyDrive/Flowers"
width=height=224
batch_size=32
data_generator=ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=10,
    zoom_range=0.1,
    validation_split=0.1
)

In [6]:
train_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training',

)

val_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Train'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='validation',

)

test_data=data_generator.flow_from_directory(
    os.path.join(dataset_path,'Test'),
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,

)

Found 986 images belonging to 17 classes.
Found 102 images belonging to 17 classes.
Found 272 images belonging to 17 classes.


In [ ]:
model_v=tf.keras.applications.VGG16(
    input_shape=(width,height,3),
    include_top=False,
    weights='imagenet'
)

In [8]:
for layer in model_v.layers:
  layer.trainable=False

In [9]:
model=tf.keras.Sequential([
        model_v,
        layers.Flatten(),
        layers.Dense(1024,activation='relu'),
        layers.Dense(128,activation='relu'),
        layers.Dense(17,activation='softmax'),
])

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project="Flowers")
config = wandb.config
config.learning_rate = 0.001

In [12]:

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= config.learning_rate ),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=['accuracy'],)

In [13]:
model.fit(train_data,
          steps_per_epoch=train_data.samples//batch_size,
          validation_data=val_data,
          validation_steps=val_data.samples/batch_size,
          epochs=10,
          callbacks=[WandbCallback()]

    
)

Epoch 1/10
30/30 [==============================] - 582s 18s/step - loss: 4.0150 - accuracy: 0.2799 - val_loss: 1.3576 - val_accuracy: 0.5980
Epoch 2/10
30/30 [==============================] - 22s 722ms/step - loss: 0.8729 - accuracy: 0.7484 - val_loss: 0.7389 - val_accuracy: 0.7549
Epoch 3/10
30/30 [==============================] - 22s 716ms/step - loss: 0.4826 - accuracy: 0.8606 - val_loss: 0.7936 - val_accuracy: 0.7941
Epoch 4/10
30/30 [==============================] - 22s 721ms/step - loss: 0.2764 - accuracy: 0.9287 - val_loss: 0.4438 - val_accuracy: 0.8529
Epoch 5/10
30/30 [==============================] - 22s 719ms/step - loss: 0.1603 - accuracy: 0.9633 - val_loss: 0.5155 - val_accuracy: 0.8627
Epoch 6/10
30/30 [==============================] - 21s 714ms/step - loss: 0.1059 - accuracy: 0.9822 - val_loss: 0.4505 - val_accuracy: 0.8627
Epoch 7/10
30/30 [==============================] - 21s 713ms/step - loss: 0.0757 - accuracy: 0.9864 - val_loss: 0.3457 - val_accuracy: 0.9118


In [14]:
model.evaluate(test_data)

9/9 [==============================] - 131s 16s/step - loss: 1.3397 - accuracy: 0.6838


[1.339702844619751, 0.6838235259056091]